In [6]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
import unicodedata
import cv2

def char_normalize(f):
    return unicodedata.normalize('NFC', f)

def get_unpadded_borders(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
    processed = cv2.morphologyEx(np.array(img), cv2.MORPH_CLOSE, kernel)
    npimg = 255 - np.array(processed)
    npimg[np.where(npimg < 16)] = 0
    wmin = npimg.sum(0).nonzero()[0].min()
    wmax = npimg.sum(0).nonzero()[0].max()
    hmin = npimg.sum(1).nonzero()[0].min()
    hmax = npimg.sum(1).nonzero()[0].max()
    return wmin, wmax, hmin, hmax

def unpad(img):
    wmin, wmax, hmin, hmax = get_unpadded_borders(img)
    wmin = wmin - 1 if wmin > 0 else wmin
    wmax = wmax + 1 if wmax < img.width else wmax
    hmin = hmin - 1 if hmin > 0 else hmin
    hmax = hmax + 1 if hmax < img.height else hmax
    return Image.fromarray(np.array(img)[hmin:hmax,wmin:wmax])

def centering(img, new_size=128, padding_color=255):

    # Resize the image using the new dimensions
    resized_img = unpad(img)
    
    # Create a new blank image with padding
    padded_img = Image.new("L", (new_size, new_size), padding_color)
    
    # Calculate the position to paste the resized image
    left = (padded_img.width - resized_img.width) // 2
    top = (padded_img.height - resized_img.height) // 2
    right = left + resized_img.width
    bottom = top + resized_img.height
    
    # Paste the resized image onto the blank image
    padded_img.paste(resized_img, (left, top, right, bottom))
    
    return padded_img


def merge_img(img0, img1, img2):
    mmg = Image.new("L", (128*2, 128*2), 255)
    mmg.paste(img0, (0,0))
    mmg.paste(img1, (128,0))
    if img2 is not None:
        mmg.paste(img2, (64,128))
    return mmg.resize((96,96), Image.BILINEAR)

def get_imgs(fd, font, letter):
    
    ch1_list = ['ㄱ', 'ㄱㄱ', 'ㄴ', 'ㄷ', 'ㄷㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅂㅂ', 'ㅅ', 'ㅅㅅ', 'ㅇ', 'ㅈ', 'ㅈㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    ch2_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    ch3_list = [None, 'ㄱ', 'ㄱㄱ', 'ㄱㅅ', 'ㄴ', 'ㄴㅈ', 'ㄴㅎ', 'ㄷ', 'ㄹ', 'ㄹㄱ', 'ㄹㅁ', 'ㄹㅂ', 'ㄹㅅ', 'ㄹㅌ', 'ㄹㅍ', 'ㄹㅎ', 'ㅁ', 'ㅂ', 'ㅂㅅ', 'ㅅ', 'ㅅㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    horizlist = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ','ㅣ']
    
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2
    
    c1 = ch1_list[ch1]
    c2 = ch2_list[ch2]
    c3 = ch3_list[ch3]
    
    if c2 in horizlist:
        if os.path.exists(f"{fd}/{font}/{font}__{c1}1.png"):
            c1 += "1"
        elif os.path.exists(f"{fd}/{font}/{font}__{c1}3.png"):
            c1 += "3"
        if os.path.exists(f"{fd}/{font}/{font}__{c2}2.png"):
            c2 += "2"
        if c3 is not None:
            if os.path.exists(f"{fd}/{font}/{font}__{c3}3.png"):
                c3 += "3"
    else:
        if os.path.exists(f"{fd}/{font}/{font}__{c1}3.png"):
            c1 += "3"
        elif os.path.exists(f"{fd}/{font}/{font}__{c1}1.png"):
            c1 += "1"
        if os.path.exists(f"{fd}/{font}/{font}__{c2}2.png"):
            c2 += "2"
        if c3 is not None:
            if os.path.exists(f"{fd}/{font}/{font}__{c3}3.png"):
                c3 += "3"

    return (
        Image.open(f"{fd}/{font}/{font}__{c1}.png"),
        Image.open(f"{fd}/{font}/{font}__{c2}.png"),
        None if c3 is None else Image.open(f"{fd}/{font}/{font}__{c3}.png"),
    )


def get_all_korean():

    def nextKorLetterFrom(letter):
        lastLetterInt = 15572643
        if not letter:
            return '가'
        a = letter
        b = a.encode('utf8')
        c = int(b.hex(), 16)

        if c == lastLetterInt:
            return False

        d = hex(c + 1)
        e = bytearray.fromhex(d[2:])

        flag = True
        while flag:
            try:
                r = e.decode('utf-8')
                flag = False
            except UnicodeDecodeError:
                c = c+1
                d = hex(c)
                e = bytearray.fromhex(d[2:])
        return e.decode()

    returns = []
    flag = True
    k = ''
    while flag:
        k = nextKorLetterFrom(k)
        if k is False:
            flag = False
        else:
            returns.append(k)
    return returns

# Raw에서 파일명정규화+이미지센터링해서 이동하기

In [2]:
rawfd = "/home/jupyter/ai_font/data/exp0717/test0730_handcut_raw"
partsfd = "/home/jupyter/ai_font/data/exp0717/test0730_handcut_parts"
wholefd = "/home/jupyter/ai_font/data/exp0717/test0730_handcut_whole"

In [3]:
font = "플레이브밤비"
os.makedirs(f"{partsfd}/{font}", exist_ok=True)
os.makedirs(f"{wholefd}/{font}", exist_ok=True)
for n, path in [(f, f"{rawfd}/{f}") for f in os.listdir(rawfd) if f.endswith(".png")]:
    nn = char_normalize(n)
    img = Image.open(path)
    centered_img = centering(img)
    if nn[-5] in ["1","2","3"]:
        centered_img.save(f"{partsfd}/{font}/{font}__{nn}")
    else:
        centered_img.save(f"{wholefd}/{font}/{font}__{nn}")

# Assembled 생성하기

In [9]:
asbfd = "/home/jupyter/ai_font/data/exp0809/test0809_handcut_assembled"
font = "플레이브밤비"
os.makedirs(f"{asbfd}/{font}", exist_ok=True)

In [10]:
ak = get_all_korean()
for n in ak:
    letter = n
    try:
        img0, img1, img2 = get_imgs(partsfd, font, letter)
        mmg = merge_img(img0, img1, img2)
        mmg.save(f"{asbfd}/{font}/{font}__{letter}.png")
    except:
        print(letter)

갃
갟
갻
걗
걳
게
겍
겎
겏
겐
겑
겒
겓
겔
겕
겖
겗
겘
겙
겚
겛
겜
겝
겞
겟
겠
겡
겢
겣
겤
겥
겦
겧
겫
곇
곣
과
곽
곾
곿
관
괁
괂
괃
괄
괅
괆
괇
괈
괉
괊
괋
괌
괍
괎
괏
괐
광
괒
괓
괔
괕
괖
괗
괛
괷
굓
굯
궋
궧
귃
귟
귻
긗
긳
깏
깫
꺇
꺣
꺿
께
껙
껚
껛
껜
껝
껞
껟
껠
껡
껢
껣
껤
껥
껦
껧
껨
껩
껪
껫
껬
껭
껮
껯
껰
껱
껲
껳
껷
꼓
꼯
꽈
꽉
꽊
꽋
꽌
꽍
꽎
꽏
꽐
꽑
꽒
꽓
꽔
꽕
꽖
꽗
꽘
꽙
꽚
꽛
꽜
꽝
꽞
꽟
꽠
꽡
꽢
꽣
꽧
꾃
꾟
꾻
꿗
꿳
뀏
뀫
끇
끣
끿
낛
낷
냓
냯
넋
네
넥
넦
넧
넨
넩
넪
넫
넬
넭
넮
넯
넰
넱
넲
넳
넴
넵
넶
넷
넸
넹
넺
넻
넼
넽
넾
넿
녃
녟
녻
놔
놕
놖
놗
놘
놙
놚
놛
놜
놝
놞
놟
놠
놡
놢
놣
놤
놥
놦
놧
놨
놩
놪
놫
놬
놭
놮
놯
놳
뇏
뇫
눇
눣
눿
뉛
뉷
늓
늯
닋
닧
댃
댟
댻
덗
데
덱
덲
덳
덴
덵
덶
덷
델
덹
덺
덻
덼
덽
덾
덿
뎀
뎁
뎂
뎃
뎄
뎅
뎆
뎇
뎈
뎉
뎊
뎋
뎏
뎫
돇
돠
돡
돢
돣
돤
돥
돦
돧
돨
돩
돪
돫
돬
돭
돮
돯
돰
돱
돲
돳
돴
돵
돶
돷
돸
돹
돺
돻
돿
됛
됷
둓
둯
뒋
뒧
듃
듟
듻
딗
딳
땏
땫
떇
떣
떼
떽
떾
떿
뗀
뗁
뗂
뗃
뗄
뗅
뗆
뗇
뗈
뗉
뗊
뗋
뗌
뗍
뗎
뗏
뗐
뗑
뗒
뗓
뗔
뗕
뗖
뗗
뗛
뗷
똓
똬
똭
똮
똯
똰
똱
똲
똳
똴
똵
똶
똷
똸
똹
똺
똻
똼
똽
똾
똿
뙀
뙁
뙂
뙃
뙄
뙅
뙆
뙇
뙋
뙧
뚃
뚟
뚻
뛗
뛳
뜏
뜫
띇
띣
띿
랛
랷
럓
럯
레
렉
렊
렋
렌
렍
렎
렏
렐
렑
렒
렓
렔
렕
렖
렗
렘
렙
렚
렛
렜
렝
렞
렟
렠
렡
렢
렣
렧
롃
롟
롸
롹
롺
롻
롼
롽
롾
롿
뢀
뢁
뢂
뢃
뢄
뢅
뢆
뢇
뢈
뢉
뢊
뢋
뢌
뢍
뢎
뢏
뢐
뢑
뢒
뢓
뢗
뢳
룏
룫
뤇
뤣
뤿
륛
륷
릓
릯
맋
맧
먃
먟
먻
메
멕
멖
멗
멘
멙
멚
멛
멜
멝
멞
멟
멠
멡
멢
멣
멤
멥
멦
멧
멨
멩
멪
멫
멬
멭
멮
멯
멳
몏
몫
뫄
뫅
뫆
뫇
뫈
뫉
뫊
뫋
뫌
뫍
뫎
뫏
뫐
뫑


# Sampling된 이미지 버킷에 압축+업로드하기

In [42]:
import os
import zipfile
from google.cloud import storage

def zip_folder(folder_path, output_path):
    """
    Zips the contents of an entire folder (including subfolders) into a zip file.
    
    :param folder_path: Path to the folder to be zipped.
    :param output_path: Path to save the output zip file.
    """
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname)
    print(f"Folder {folder_path} zipped into {output_path}.")

def upload_file_to_bucket(bucket_name, source_file_path, destination_blob_name):
    """
    Uploads a file to the Google Cloud Storage bucket.
    
    :param bucket_name: Name of the GCP bucket.
    :param source_file_path: Path to the file to be uploaded.
    :param destination_blob_name: Name of the blob in the bucket.
    """
    # Create a client
    storage_client = storage.Client()

    # Get the bucket
    bucket = storage_client.bucket(bucket_name)

    # Create a blob (file in GCP bucket)
    blob = bucket.blob(destination_blob_name)

    # Upload the file
    blob.upload_from_filename(source_file_path)
    print(f"File {source_file_path} uploaded to {destination_blob_name} in bucket {bucket_name}.")

folder_to_zip = "/home/jupyter/ai_font/data/exp0717/report0730_plain"
zip_file_name = "report0730_plain.zip"
bucket_name = "leo_font"
destination_blob_name = "exp0717/report/report0730_plain.zip"  # Path in the bucket

# Step 1: Zip the folder
zip_folder(folder_to_zip, zip_file_name)

# Step 2: Upload the ZIP file to the GCP bucket
upload_file_to_bucket(bucket_name, zip_file_name, destination_blob_name)

Folder /home/jupyter/ai_font/data/exp0717/report0730_plain zipped into report0730_plain.zip.
File report0730_plain.zip uploaded to exp0717/report/report0730_plain.zip in bucket leo_font.


# Test 이미지 버킷에 올리기

In [43]:
name = "test0730_handcut_whole"
folder_to_zip = f"/home/jupyter/ai_font/data/exp0717/{name}"
zip_file_name = f"{name}.zip"
bucket_name = "leo_font"
destination_blob_name = f"exp0717/report/{name}.zip"  # Path in the bucket

# Step 1: Zip the folder
zip_folder(folder_to_zip, zip_file_name)

# Step 2: Upload the ZIP file to the GCP bucket
upload_file_to_bucket(bucket_name, zip_file_name, destination_blob_name)

Folder /home/jupyter/ai_font/data/exp0717/test0730_handcut_whole zipped into test0730_handcut_whole.zip.
File test0730_handcut_whole.zip uploaded to exp0717/report/test0730_handcut_whole.zip in bucket leo_font.
